so the point of this one is to get a status check on our current follows, comparing them to last week's follows. If we find any follows that were previously there and are not we need to check to see if 1. that user exists and wasn't just banned/deleted, and 2. if we are following that user if they exist, and if 1 and 2 then unfollow them.

In [12]:
pip install boto3

In [13]:
pip install atproto

In [14]:
from atproto import Client
from google.colab import userdata
# aws stuff
import boto3
from botocore.exceptions import ClientError
# json necessary to parse secret string, and write/read s3 objects
import json
# these imports are to use github apis to do logging, base64 is to parse the json
import requests
import base64
# datetime for logging
import datetime
import zoneinfo

In [15]:
AWS_KEY = userdata.get('aws_access_key')
AWS_SECRET_KEY = userdata.get('aws_secret_access_key')
REGION = userdata.get('aws_region')
SECRETS_ID = userdata.get('aws_secretsmanager_id')

# this file doesn't need ddb access
# DDB = 'dynamodb'
S3 = 's3'
# DDB_TABLE = 'rickybot-ddb'
S3_BUCKET = 'rickybot-s3'
S3_KEY = 'STATUS' # unlike the others that have a key determined by the day of the week, this will check the same spot every time.


USER_TIMEZONE = "US/Eastern"
FILE_PATH = "LOGGING_STATUS_02.txt"
BRANCH = "main"

In [16]:
# connect to aws
try:
  aws_session = boto3.Session(
          aws_access_key_id = AWS_KEY,
          aws_secret_access_key = AWS_SECRET_KEY,
          region_name = REGION
      )
except:
  print('failed to begin AWS session')
  # return with error
  # this is the only error that we can't log to github, because we never got the credentials

In [17]:
# then connect to secrets manager
try:
  secrets_client = aws_session.client('secretsmanager')
  secret_value = secrets_client.get_secret_value(SecretId=SECRETS_ID)
  secret_string = secret_value['SecretString']
  secret_map = json.loads(secret_string)
except:
  print('failed to reach aws secrets manager')
  # return with error

In [18]:
# create constants from the values in the secrets manager
BSKY_USERNAME = secret_map['bsky_username']
BSKY_PASSWORD = secret_map['bsky_password']
GITHUB_TOKEN = secret_map['github_token']
GITHUB_REPO = secret_map['github_user/repo']
HUGGING_TOKEN = secret_map['hugging_token']

In [28]:
# before the program starts let's set up the logging function so we can insert it at any point where our program could break
def logging_status(logging_text):
  # LOGGING ALL THE CHANGES TO OUR LOGGING FILE IN GITHUB
  cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))
  datetime_object = datetime.datetime.fromtimestamp(cur_timestamp.timestamp())
  date_only = str(datetime_object.date())
  commit_message = "Logging status of followers on " + date_only


  # Step 1: Get the file's current content and SHA
  url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{FILE_PATH}"
  headers = {"Authorization": f"token {GITHUB_TOKEN}"}
  response = requests.get(url, headers=headers)
  response_json = response.json()

  # Decode the content of the file
  file_sha = response_json["sha"]
  content = base64.b64decode(response_json["content"]).decode("utf-8")

  # Step 2: Modify the file content
  new_content = content + date_only + ': ' + logging_text + '\n'
  encoded_content = base64.b64encode(new_content.encode("utf-8")).decode("utf-8")

  # Step 3: Push the updated content
  data = {
      "message": commit_message,
      "content": encoded_content,
      "sha": file_sha,
      "branch": BRANCH,
  }
  update_response = requests.put(url, headers=headers, json=data)

  if update_response.status_code == 200:
      print("Logging file updated successfully! Here's what was added to the logs:")
      print(date_only + ": " + logging_text)
  else:
      print(f"Error: {update_response.json()}")

In [20]:
# initialize the s3 client and get the bucket
try:
  s3 = aws_session.client(S3)
  buckets = s3.list_buckets()
  bucket = s3.list_objects_v2(Bucket=S3_BUCKET)
except:
  print('ERROR - failed to get s3 bucket')
  logging_status('ERROR - failed to get s3 bucket')
  # return with error

In [21]:
# log in to bluesky
try:
  client = Client()
  client.login(BSKY_USERNAME, BSKY_PASSWORD)
except:
  print('ERROR - failed to log in to bluesky')
  logging_status('ERROR - failed to log in to bluesky')

In [22]:
# get the current list of bluesky followers we'll create a map of follower_id: follow_uri, and follow_uri will identify if we follow them
try:
  current_followers = {}
  followers_count = client.get_profile(actor=BSKY_USERNAME).followers_count
  remaining = followers_count
  next_page = ''
  while remaining > 0:
    print (f'starting the next page of follows. {remaining} followers remaining to check.')
    cur_limit = min(remaining, 100)
    remaining -= cur_limit
    followers = client.get_followers(actor=BSKY_USERNAME, cursor= next_page, limit=cur_limit)
    next_page = followers.cursor
    followers = followers.followers
    for user in followers:
      did = user.did
      handle = user.handle
      follow_uri = user.viewer.following
      # print(handle, did, follow_uri)
      # handles can change we only want to deal with the did
      current_followers[handle] = follow_uri
except:
  print('ERROR - failed to gather followers')
  logging_status('ERROR - failed to gather followers')
  # return early


starting the next page of follows. 9738 followers remaining to check.
starting the next page of follows. 9638 followers remaining to check.
starting the next page of follows. 9538 followers remaining to check.
starting the next page of follows. 9438 followers remaining to check.
starting the next page of follows. 9338 followers remaining to check.
starting the next page of follows. 9238 followers remaining to check.
starting the next page of follows. 9138 followers remaining to check.
starting the next page of follows. 9038 followers remaining to check.
starting the next page of follows. 8938 followers remaining to check.
starting the next page of follows. 8838 followers remaining to check.
starting the next page of follows. 8738 followers remaining to check.
starting the next page of follows. 8638 followers remaining to check.
starting the next page of follows. 8538 followers remaining to check.
starting the next page of follows. 8438 followers remaining to check.
starting the next pa

In [33]:
# now we get the old list of our followers from s3
# then if we find an old list in our bucket we compare it to the new list and unfollow anyone who is not in our new list.
count_removed = 0
count_failed_removal = 0
try:
  s3.head_object(Bucket=S3_BUCKET, Key=S3_KEY)
  print("Object existed in s3 bucket.")
  response = s3.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
  # creates a list from the json info in the s3 bucket
  old_followers = json.loads(response["Body"].read())
  print(old_followers, type(old_followers))
  for user_did, follow_uri in old_followers.items():
    if user_did not in current_followers:
      print('not in followers', user_did)
      try:
        client.delete_follow(follow_uri)
        count_removed += 1
        # print(cur)
      except Exception as e:
        # print(f'failed on {i} - uri: {follow_uri} \n {e}')
        count_failed_removal +=1
  follow_diff = len(current_followers) - len(old_followers)
  logging_status(f'successfully compared followers - followers are {"up" if follow_diff >= 0 else "down"} {abs(follow_diff)} this week. {count_removed + count_failed_removal} users stopped following. {count_removed} were successfully unfollowed, with {count_failed_removal} failures.')
except s3.exceptions.ClientError as e:
  if e.response["Error"]["Code"] == "404": # object was not found at the key
    print("ERROR - there was no previous followers list to compaare to found in the s3 bucket. Adding the new list of followers.")
    logging_status("ERROR - there was no previous followers list to compare to found in the s3 bucket. Adding the new list of followers.")
  else:
    print("ERROR - failed to get previous followers list from s3 bucket")
    logging_status("ERROR - failed to get previous followers list from s3 bucket")
finally:
  # regardless of if we were able to perform the comparison or not we want to input our new list into storage
  try:
    s3.put_object(
        Bucket=S3_BUCKET,
        Key=S3_KEY,
        Body=json.dumps(current_followers),
        ContentType="application/json"
    )
    logging_status(f'successfully uploaded current followers as map to s3.')
  except Exception as e:
      print(f"ERROR - failed to upload new followers map to s3: {e}")
      logging_status(f"ERROR - failed to upload new followers map to s3: {e}")


Object existed in s3 bucket.
{'liliput1990.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3perw542p', 'macknowles.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3fchl7z25', 'gottaskate8.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3uympdy26', 'vancityreynolds12.bsky.social': None, 'arlnee.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3qclods2e', 'deakat.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3pjp7aa26', 'meofetti.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3nn7b6i2l', 'jld1966.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3pfuzm52j', 'erihu73.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3j32k2x2h', 'sproutastic.bsky.social': 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lig3qbjegn2j', 'cheyenne77.bsky.